In [ ]:
!wget https://www.dropbox.com/s/aabqnsc1mmh99kv/teacher0to4.pt
!wget https://www.dropbox.com/s/tl4jpuuzth5rpyh/teacher5to9.pt
!wget https://www.dropbox.com/s/h7l845py2d3t6o5/data.zip
!wget https://www.dropbox.com/s/fxpak3osyaj0i4r/u_methods_v2.csv
!unzip data.zip

--2021-06-08 12:24:21--  https://www.dropbox.com/s/aabqnsc1mmh99kv/teacher0to4.pt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/aabqnsc1mmh99kv/teacher0to4.pt [following]
--2021-06-08 12:24:21--  https://www.dropbox.com/s/raw/aabqnsc1mmh99kv/teacher0to4.pt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc98009a3b1cac523280b2df725d.dl.dropboxusercontent.com/cd/0/inline/BQAJ8xHQoHPWUySy8BL8jFQar_ezaQAp12kDcU-bKZI4bHCW1L6GM3YvwKFxaWSCx_Y2fvi19K01xkRXLgCZmjM_z5uwV1Cf61iv6bo6NvaL41Seugr--9Ck-FVKybEpHuIzsLcJtXT6gAxUHwYd3VEO/file# [following]
--2021-06-08 12:24:21--  https://uc98009a3b1cac523280b2df725d.dl.dropboxusercontent.com/cd/0/inline/BQAJ8xHQoHPWUySy8BL8jFQar_ezaQAp12kDcU-bKZI4bHCW1L6GM3YvwKFxaWSCx_Y2fvi19K01xkRXL

# LOADING MODEL AND MNIST CLASSES

In [ ]:
import os
import time
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Import torchvision functions/classes for MNIST import and data loaders
import torchvision
import torchvision.transforms as transforms


class Model(nn.Module):

    def __init__(self, n_classes, hidden_size=1200, dropout=0.0, hidden_dropout=0.0):
        super(Model, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden1 = nn.Linear(784, hidden_size, bias=True)
        self.hidden1_dropout = nn.Dropout(hidden_dropout)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.hidden2_dropout = nn.Dropout(hidden_dropout)
        self.hidden3 = nn.Linear(hidden_size, n_classes, bias=True)

    def forward(self, x):

        x = self.dropout(x)
        x = F.relu(self.hidden1(x))
        x = self.hidden1_dropout(x)
        x = F.relu(self.hidden2(x))
        x = self.hidden2_dropout(x)
        x = self.hidden3(x)
        return x



class MnistDataset(Dataset):
    def __init__(self, data, target, transformation=None):
        self.images = data
        self.targets = target
        self.transformation = transforms.Compose([
              transforms.RandomAffine(0, (1/14, 1/14)),
              transforms.Normalize((0.5,), (0.5,))
            ])

    def __len__(self):
        return len(self.images)


    def __getitem__(self, idx):
        return self.images[idx], self.targets[idx]


class MnistQs():
      def __init__(self):
          self.df = pd.read_pickle("u_methods_v2.csv")
          self.u_CE = self.df["u_CE"]
          self.u_MFPS = self.df["u_MFPS"]
          self.u_MFLS = self.df["u_MFLS"]

      def __len__(self):
          return self.df.shape[0]

      def __getitem__(self, idx):
          return self.u_CE.iloc[idx], self.u_MFPS.iloc[idx], self.u_MFLS.iloc[idx]

      def get_u_CE(self):
          self.u_CE = torch.zeros(60000, 10)

          for idx, u in enumerate(self.df.u_CE):
              self.u_CE[idx, :] = torch.tensor(u)

          return self.u_CE

      def get_u_MFPS(self):
          self.u_MFPS = torch.zeros(60000, 10)

          for idx, u in enumerate(self.df.u_MFPS):
              self.u_MFPS[idx, :] = torch.tensor(u)
          
          return self.u_MFPS

      def get_u_MFLS(self):
          self.u_MFLS = torch.zeros(60000, 10)

          for idx, u in enumerate(self.df.u_MFLS):
              self.u_MFLS[idx, :] = torch.tensor(u)
          
          return self.u_MFLS


# LOAD THE DATASET AND THE TEACHER FILES

In [ ]:
teacher1 = torch.load("teacher0to4.pt")
teacher1.to("cuda")
teacher2 = torch.load("teacher5to9.pt")
teacher2.to("cuda")

Model(
  (dropout): Dropout(p=0.2, inplace=False)
  (hidden1): Linear(in_features=784, out_features=1200, bias=True)
  (hidden1_dropout): Dropout(p=0.5, inplace=False)
  (hidden2): Linear(in_features=1200, out_features=1200, bias=True)
  (hidden2_dropout): Dropout(p=0.5, inplace=False)
  (hidden3): Linear(in_features=1200, out_features=5, bias=True)
)

# IMPLEMENTING THE STUDENT

In [ ]:
60000/125

480.0

In [ ]:
images, targets = torch.load("/content/data/MNIST/processed/training.pt")
trainset = MnistDataset(images, targets)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=125, shuffle=False, num_workers=2)
mnistq = MnistQs()

In [ ]:
student_CE = Model(n_classes=10, hidden_size=500, dropout=0.1, hidden_dropout=0.1)
student_CE.to("cuda")

criterion = nn.KLDivLoss()
m = nn.Softmax(dim=0)
learning_rate = 0.001
epochs = 50

# Set up loss function and optimizer
optimizer = optim.SGD(student_CE.parameters(), lr=learning_rate, momentum=0.9)

# Training student with method 1

In [ ]:
# Run over 1000 epochs (1 epoch = visited all items in dataset)
for epoch in range(epochs):
    running_loss = 0.0
    total = 0

    for image, qs in zip(trainloader, np.array_split(mnistq.get_u_CE(), 480)):
        # Apply the learning rate decay
        if(epoch % 100 == 0 and epoch != 0):
            learning_rate = learning_rate * 0.5
            optimizer = optim.SGD(student_CE.parameters(), lr= learning_rate, momentum=0.9)
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, targets = image
        inputs = torch.flatten(inputs, start_dim=1).to("cuda")
        qs = qs.to("cuda")
        # target = labels.to("cuda").long()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # Set temperature and the weights for losses linear combination
        w = 0.7
        T = 3

        # Student forward + backward + optimize
        logits_student = student_CE(inputs.float())
        # loss = torch.cdist(m(qs), m(logits_student), p=2)
        loss = criterion(F.log_softmax(qs/T, dim=1), F.softmax(logits_student/T, dim=1))
        loss.backward()
        optimizer.step()

        total += len(image)

        # print statistics
        running_loss += loss.item()
    # print every epoch
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / total))

print('Finished Training')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2611: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."


[1] loss: 0.029
[2] loss: 0.014
[3] loss: 0.011
[4] loss: 0.010
[5] loss: 0.009
[6] loss: 0.008
[7] loss: 0.008
[8] loss: 0.008
[9] loss: 0.007
[10] loss: 0.007
[11] loss: 0.007
[12] loss: 0.007
[13] loss: 0.007
[14] loss: 0.007
[15] loss: 0.007
[16] loss: 0.007
[17] loss: 0.006
[18] loss: 0.006
[19] loss: 0.006
[20] loss: 0.006
[21] loss: 0.006
[22] loss: 0.006
[23] loss: 0.006
[24] loss: 0.006
[25] loss: 0.006
[26] loss: 0.006
[27] loss: 0.006
[28] loss: 0.006
[29] loss: 0.006
[30] loss: 0.006
[31] loss: 0.006
[32] loss: 0.006
[33] loss: 0.006
[34] loss: 0.006
[35] loss: 0.006
[36] loss: 0.006
[37] loss: 0.006
[38] loss: 0.006
[39] loss: 0.005
[40] loss: 0.005
[41] loss: 0.005
[42] loss: 0.005
[43] loss: 0.005
[44] loss: 0.005
[45] loss: 0.005
[46] loss: 0.005
[47] loss: 0.005
[48] loss: 0.005
[49] loss: 0.005
[50] loss: 0.005
Finished Training


# Testing student with method 1

In [ ]:
images_test, targets_test = torch.load("/content/data/MNIST/processed/test.pt")
testset = MnistDataset(images_test, targets_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=True, num_workers=2)

In [ ]:
# Define support function used to convert label to one-hot encoded tensor
def convert_labels(labels):
    target = torch.zeros([len(labels), 10], dtype=torch.float32)
    for i, l in enumerate(labels):
      target[i][l] = 1.0
    return target

# Run model on test set and determine accuracy
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to("cuda")
        target = convert_labels(labels).to("cuda")
        outputs = student_CE(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        _, target = torch.max(target.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
        # for i, val in enumerate(predicted):
        #   wrong[target[i]][val] += 1

# Output model accuracy to user
print('Accuracy of the network on test images: %d %% (%d wrong out of %d)' % (
    100 * correct / total, total - correct, total))

tensor(8, device='cuda:0')
Accuracy of the network on test images: 74 % (2552 wrong out of 10000)


In [ ]:
classes = np.arange(0, 10)

# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to("cuda")
        target = convert_labels(labels).to("cuda")
        outputs = student_CE(inputs.float())
        _, predictions = torch.max(outputs, 1)
        _, target = torch.max(target.data, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(target, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

print(correct_pred)
print(total_pred)


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f"Accuracy for {str(classname)} is: {round(accuracy, 4)} %")


{0: 869, 1: 915, 2: 687, 3: 848, 4: 741, 5: 518, 6: 771, 7: 836, 8: 780, 9: 614}
{0: 980, 1: 1135, 2: 1032, 3: 1010, 4: 982, 5: 892, 6: 958, 7: 1028, 8: 974, 9: 1009}
Accuracy for 0 is: 88.6735 %
Accuracy for 1 is: 80.6167 %
Accuracy for 2 is: 66.5698 %
Accuracy for 3 is: 83.9604 %
Accuracy for 4 is: 75.4582 %
Accuracy for 5 is: 58.0717 %
Accuracy for 6 is: 80.4802 %
Accuracy for 7 is: 81.323 %
Accuracy for 8 is: 80.0821 %
Accuracy for 9 is: 60.8523 %


# Training student with method 2 (MFPS)

In [ ]:
student_MFPS = Model(n_classes=10, hidden_size=500, dropout=0.1, hidden_dropout=0.1)
student_MFPS.to("cuda")

# bce_with_logits = torch.nn.BCEWithLogitsLoss()
learning_rate = 0.001
epochs = 20
m = nn.Softmax(dim=1)

# Set up loss function and optimizer
optimizer = optim.SGD(student_MFPS.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(epochs):
    running_loss = 0.0
    total = 0

    for image, qs in zip(trainloader, np.array_split(mnistq.get_u_MFPS(), 480)):
        # Apply the learning rate decay
        if(epoch % 100 == 0 and epoch != 0):
            learning_rate = learning_rate * 0.5
            optimizer = optim.SGD(student_MFPS.parameters(), lr= learning_rate, momentum=0.9)
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, targets = image
        inputs = torch.flatten(inputs, start_dim=1).to("cuda")
        
        qs = qs.to("cuda")
        # target = labels.to("cuda").long()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # Set temperature and the weights for losses linear combination
        w = 0.7
        T = 2

        # Student forward + backward + optimize
        logits_student = student_MFPS(inputs.float())
        # loss = criterion(F.log_softmax(qs/T, dim=1), F.softmax(logits_student/T, dim=1))
        loss = criterion(qs, m(logits_student/T))
        loss.backward()
        optimizer.step()

        total += len(image)

        # print statistics
        running_loss += loss.item()
    # print every epoch
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / total))

print('Finished Training')

# Testing student with method 2 (MFPS)

In [ ]:
# Define support function used to convert label to one-hot encoded tensor
def convert_labels(labels):
    target = torch.zeros([len(labels), 10], dtype=torch.float32)
    for i, l in enumerate(labels):
      target[i][l] = 1.0
    return target

# Run model on test set and determine accuracy
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to("cuda")
        target = convert_labels(labels).to("cuda")
        outputs = student_MFPS(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        _, target = torch.max(target.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
        # for i, val in enumerate(predicted):
        #   wrong[target[i]][val] += 1

# Output model accuracy to user
print('Accuracy of the network on test images: %d %% (%d wrong out of %d)' % (
    100 * correct / total, total - correct, total))

Accuracy of the network on test images: 33 % (6675 wrong out of 10000)


# Training student with method 2 (MFLS)

In [ ]:
student_MFLS = Model(n_classes=10, hidden_size=500, dropout=0.1, hidden_dropout=0.1)
student_MFLS.to("cuda")

criterion = nn.KLDivLoss()
m = nn.Softmax(dim=0)
learning_rate = 0.001
epochs = 100

# Set up loss function and optimizer
optimizer = optim.SGD(student_MFLS.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(epochs):
    running_loss = 0.0
    total = 0

    for image, qs in zip(trainloader, np.array_split(mnistq.get_u_MFLS(), 480)):
        # Apply the learning rate decay
        if(epoch % 100 == 0 and epoch != 0):
            learning_rate = learning_rate * 0.5
            optimizer = optim.SGD(student_MFLS.parameters(), lr= learning_rate, momentum=0.9)
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, targets = image
        inputs = torch.flatten(inputs, start_dim=1).to("cuda")
        qs = qs.to("cuda")
        # target = labels.to("cuda").long()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # Set temperature and the weights for losses linear combination
        w = 0.7
        T = 15

        # Student forward + backward + optimize
        logits_student = student_MFLS(inputs.float())
        loss = criterion(F.log_softmax(qs/T, dim=1), F.softmax(logits_student/T, dim=1))
        loss.backward()
        optimizer.step()

        total += len(image)

        # print statistics
        running_loss += loss.item()
    # print every epoch
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / total))

print('Finished Training')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2611: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."


[1] loss: 0.039
[2] loss: 0.025
[3] loss: 0.023
[4] loss: 0.022
[5] loss: 0.021
[6] loss: 0.020
[7] loss: 0.020
[8] loss: 0.019
[9] loss: 0.019
[10] loss: 0.018
[11] loss: 0.018
[12] loss: 0.018
[13] loss: 0.018
[14] loss: 0.017
[15] loss: 0.017
[16] loss: 0.017
[17] loss: 0.017
[18] loss: 0.017
[19] loss: 0.016
[20] loss: 0.016
[21] loss: 0.016
[22] loss: 0.016
[23] loss: 0.016
[24] loss: 0.016
[25] loss: 0.016
[26] loss: 0.016
[27] loss: 0.016
[28] loss: 0.016
[29] loss: 0.015
[30] loss: 0.015
[31] loss: 0.015
[32] loss: 0.015
[33] loss: 0.015
[34] loss: 0.015
[35] loss: 0.015
[36] loss: 0.015
[37] loss: 0.015
[38] loss: 0.015
[39] loss: 0.015
[40] loss: 0.015
[41] loss: 0.015
[42] loss: 0.015
[43] loss: 0.015
[44] loss: 0.015
[45] loss: 0.015
[46] loss: 0.015
[47] loss: 0.014
[48] loss: 0.014
[49] loss: 0.014
[50] loss: 0.014
[51] loss: 0.014
[52] loss: 0.014
[53] loss: 0.014
[54] loss: 0.014
[55] loss: 0.014
[56] loss: 0.014
[57] loss: 0.014
[58] loss: 0.014
[59] loss: 0.014
[60] l

# Testing student with method 2 MFLS

In [ ]:
# Define support function used to convert label to one-hot encoded tensor
def convert_labels(labels):
    target = torch.zeros([len(labels), 10], dtype=torch.float32)
    for i, l in enumerate(labels):
      target[i][l] = 1.0
    return target

# Run model on test set and determine accuracy
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to("cuda")
        target = convert_labels(labels).to("cuda")
        outputs = student_MFLS(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        _, target = torch.max(target.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
        # for i, val in enumerate(predicted):
        #   wrong[target[i]][val] += 1

# Output model accuracy to user
print('Accuracy of the network on test images: %d %% (%d wrong out of %d)' % (
    100 * correct / total, total - correct, total))

Accuracy of the network on test images: 82 % (1792 wrong out of 10000)


In [ ]:
classes = np.arange(0, 10)

# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to("cuda")
        target = convert_labels(labels).to("cuda")
        outputs = student_MFLS(inputs.float())
        _, predictions = torch.max(outputs, 1)
        _, target = torch.max(target.data, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(target, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

print(correct_pred)
print(total_pred)


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f"Accuracy for {str(classname)} is: {round(accuracy, 4)} %")

{0: 967, 1: 1100, 2: 781, 3: 833, 4: 460, 5: 493, 6: 705, 7: 969, 8: 916, 9: 968}
{0: 980, 1: 1135, 2: 1032, 3: 1010, 4: 982, 5: 892, 6: 958, 7: 1028, 8: 974, 9: 1009}
Accuracy for 0 is: 98.6735 %
Accuracy for 1 is: 96.9163 %
Accuracy for 2 is: 75.6783 %
Accuracy for 3 is: 82.4752 %
Accuracy for 4 is: 46.8432 %
Accuracy for 5 is: 55.2691 %
Accuracy for 6 is: 73.5908 %
Accuracy for 7 is: 94.2607 %
Accuracy for 8 is: 94.0452 %
Accuracy for 9 is: 95.9366 %
